In [ ]:
# connect 5 tools: weather + kg frus rag + convert time + arxiv + duckduckgo
# allow daily conversation beyond tool use (+ memory)

# todo:
# 1- add conversation memory to agent (make sure it chats beyond tool-use)
# 2- add rag
# 3- add serpapi

In [1]:
import os
import ast
from dotenv import load_dotenv
import arxiv

from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [4]:
chat_model = ChatOpenAI(stop=["Observation:"])

In [ ]:
from langchain_core.documents import Document
from langchain_chroma import Chroma

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

file_path = (
    "../../eth_courses/thesis/gokberk_thesis_report_04_06_23.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(pages)

vectorstore = Chroma.from_documents(
    all_splits,
    embedding=HuggingFaceEmbeddings(),
)

prompt = """System: You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Context: {context}

Human: {input}
AI:
"""

contextualize_q_system_prompt = """Given a chat history and the latest user question which might reference context in the chat history, 
formulate a standalone question which can be understood without the chat history. 
Do NOT answer the question, just reformulate it if needed and otherwise return it as is.
If chat history is empty (''), do NOT change anything in the question.

Chat history:
{history}
Question:
{input}
Reformulated Question:
"""

In [5]:
def arxiv_search(args):
    # Construct the default API client.
    client = arxiv.Client()

    # Search for the paper with ID
    query = args['query'] if 'query' in args else ""
    id_list = [args['id']] if 'id' in args else []
    
    search_by_id = arxiv.Search(query=query,
                                id_list=id_list)
    # Reuse client to fetch the paper, then print its title.
    first_result = next(client.results(search_by_id))
    
    return first_result.title

def convert_time(args):

    temp = args['time'].split(':')

    return int(temp[0])*3600 + int(temp[1])*60 + int(temp[2])


def kgfrus_search(args):

    global history

    rephrased_q = chat_model.invoke(contextualize_q_system_prompt.format(input=args['question'],history=history)).content
    print(rephrased_q)

    context = "\n\n".join([doc.page_content for doc in vectorstore.similarity_search(rephrased_q)])

    prompt_after_formatting = prompt.format(input=rephrased_q,
                                            context=context)
    print(prompt_after_formatting)
    output = chat_model.invoke(prompt_after_formatting)

    history += f"Human: {args['question']}\nAI: {output.content}\n"

    return output.content


In [ ]:
system_prompt1 = """Answer the following question as best you can. You have access to the following tools:

{tools}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do. only one action at a time.
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer.

Conversation History: 
{history}
Question: {input}
Thought:{agent_scratchpad}"""

tools = {
    'convert_time': 'A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}',
    'arxiv_search': 'A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}'
}

In [6]:
system_prompt = """You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

{tools}

To use a tool, you MUST use the following format:

Question: the input question you must answer
Thought: Do I need to use a tool? Yes. (only one action at a time.)
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)


When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

Thought: Do I need to use a tool? No.
Final Answer: [your response here]
I remind you to ALWAYS use the exact characters `Final Answer:` when you provide an answer.

Begin!

Conversation History: 
{history}
Question: {input}
Thought:{agent_scratchpad}
"""

tools = {
    'convert_time': 'A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}',
    'arxiv_search': 'A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}'
}

In [7]:
history = ''
agent_scratchpad = ''

def ask(question):

    global history, agent_scratchpad

    tools_str = ''

    for func, desc in tools.items():
        tools_str += f"{func} : {desc}\n"
    
    prompt_after_formatting = system_prompt.format(input=question, 
                                                   agent_scratchpad=agent_scratchpad, 
                                                   tools=tools_str, 
                                                   tool_names=list(tools.keys()),
                                                   history=history)
    print(prompt_after_formatting)
    output = chat_model.invoke(prompt_after_formatting)
    print(output.content)
    
    # keep reasoning
    while output.content.find("Final Answer: ")==-1:

        agent_scratchpad += output.content

        function_name = output.content.split("Action: ")[1].split('\n')[0]
        function_args = ast.literal_eval(output.content.split("Action Input: ")[1].split('\n')[0])

        result = globals()[function_name](function_args)
        agent_scratchpad += f"Observation: {result}\n"
        print(f"Observation: {result}\n")
        
        prompt_after_formatting = system_prompt.format(input=question, 
                                                    agent_scratchpad=agent_scratchpad, 
                                                    tools=tools_str, 
                                                    tool_names=list(tools.keys()),
                                                    history=history)

        output = chat_model.invoke(prompt_after_formatting)
        print(output.content)
    
    # store question and answer in memory
    history += f"Human: {question}\nAI: {output.content.split('Final Answer: ')[1]}\n"
    
    # empty scratchpad for next question
    agent_scratchpad = ''

    return output.content

In [8]:
ask("First answer how many seconds in 4:00:01.")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
arxiv_search : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}


To use a tool, you MUST use the following format:

Question: the input question you must answer
Thought: Do I need to use a tool? Yes. (only one action at a time.)
Action: the action to take, should be one of ['convert_time', 'arxiv_search']
Action Input: the input to the action
Observation: the result of th

'Thought: Do I need to use a tool? No.\nFinal Answer: The conversion of 4:00:01 to seconds is 14401.'

In [9]:
ask("Then answer What's the paper 1605.08386 about?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
arxiv_search : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}


To use a tool, you MUST use the following format:

Question: the input question you must answer
Thought: Do I need to use a tool? Yes. (only one action at a time.)
Action: the action to take, should be one of ['convert_time', 'arxiv_search']
Action Input: the input to the action
Observation: the result of th

'Thought: Do I need to use a tool? No.\nFinal Answer: Heat-bath random walks with Markov bases.'

In [10]:
ask("What's the previous paper about?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
arxiv_search : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}


To use a tool, you MUST use the following format:

Question: the input question you must answer
Thought: Do I need to use a tool? Yes. (only one action at a time.)
Action: the action to take, should be one of ['convert_time', 'arxiv_search']
Action Input: the input to the action
Observation: the result of th

'Thought: Do I need to use a tool? No.\nFinal Answer: The previous paper is about "Heat-bath random walks with Markov bases."'

In [11]:
ask("where is london located according to arxiv?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
arxiv_search : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}


To use a tool, you MUST use the following format:

Question: the input question you must answer
Thought: Do I need to use a tool? Yes. (only one action at a time.)
Action: the action to take, should be one of ['convert_time', 'arxiv_search']
Action Input: the input to the action
Observation: the result of th

'Do I need to use a tool? No.\nFinal Answer: London is located in the United Kingdom.'

In [12]:
ask("what did i ask you recently?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
arxiv_search : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}


To use a tool, you MUST use the following format:

Question: the input question you must answer
Thought: Do I need to use a tool? Yes. (only one action at a time.)
Action: the action to take, should be one of ['convert_time', 'arxiv_search']
Action Input: the input to the action
Observation: the result of th

'Do I need to use a tool? No.\nFinal Answer: You asked me where London is located according to arxiv.'

In [17]:
from duckduckgo_search import DDGS

results = DDGS().text("fatih terim age", max_results=5)

for r in results:
    print(r['body'])

Fatih Terim (born 4 September 1953) is a Turkish professional football manager and former player. He was most recently the manager of Greek Super League club Panathinaikos.. During his football career, he played for Ceyhanspor, Adana Demirspor and Galatasaray. In his eleven years of playing time for Galatasaray, he won the Prime Minister's Cup in 1975 and in 1979, the Turkish Cup in 1975-1976 ...
This is the profile site of the manager Fatih Terim. The site lists all clubs he coached and all clubs he played for. ... Date of birth/Age: Sep 4, 1953 (70) Place of Birth: Adana : Citizenship: Türkiye : Avg. term as coach: 2.15 Years: Coaching Licence: UEFA Pro Licence ...
"I am always the same Fatih Terim, from when I arrived at Galatasaray 50 years ago to now." Terim signed for Galatasaray as a midfielder, but shortly after joining was moved into defence.
Date of birth/Age: Sep 4, 1953 (70) Place of birth ... Fatih Terim is the brother of Taner Terim (Retired). More news . Overview of top 

In [19]:
import requests

r= requests.get('http://api.openweathermap.org/data/2.5/weather?q=London,uk&APPID=1ea6383225e0a528062ba60887ef2491')

In [20]:
r.content

b'{"coord":{"lon":-0.1257,"lat":51.5085},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04d"}],"base":"stations","main":{"temp":296.65,"feels_like":296.2,"temp_min":294.87,"temp_max":297.34,"pressure":1009,"humidity":44,"sea_level":1009,"grnd_level":1005},"visibility":10000,"wind":{"speed":7.2,"deg":250},"clouds":{"all":100},"dt":1719505721,"sys":{"type":2,"id":2075535,"country":"GB","sunrise":1719459920,"sunset":1719519702},"timezone":3600,"id":2643743,"name":"London","cod":200}'